In [137]:
import sys
import pandas as pd
import numpy as np
import math
import json
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy import spatial
from scipy import stats
sys.path.append("../../")
import dartlab
from importlib import reload
import mfc
%matplotlib inline
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from pymer4.models import Lmer
import pyperclip
import sklearn as sk
import simcse
import logging
# smodel = simcse.SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")
# smodel = simcse.SimCSE("princeton-nlp/sup-simcse-roberta-large") 
smodel = simcse.SimCSE("princeton-nlp/sup-simcse-bert-large-uncased")
# simcse = reload(simcse)
# sk = reload(sk)
# dartlab = reload(dartlab)

Downloading:   0%|          | 0.00/253 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [174]:
sentences_a = ["Republican insults Democrat"]
sentences_b = ["A Democrat is insulting a Republican", "A Republican is insulting a Democrat"]

sentences_a = ["Nancy Pelosi"]
sentences_b = ["Democrat", "Republican"]
similarities = smodel.similarity(sentences_a, sentences_b)
similarities

# bestMatch("BREAKING: Trump Just Gave Every WORKING, LEGAL Man Woman And Child In The USA $612 EACH!")
embeddings = smodel.encode("A woman is reading.")
embeddings

100%|██████████| 1/1 [00:00<00:00,  4.59it/s]


tensor([ 0.0019, -0.0093, -0.0222,  ...,  0.0266, -0.0588,  0.0007])

## S1.AB.2 Stimuli

In [71]:
with open('S1.AlignmentBelievability/S1.ab.2/stimuli.json') as file:
    s1ab2 = json.load(file)
    
# with open('S1.AlignmentBelievability/S1.ab.1/S1.AlignmentBelievability_content.json') as file:
#     s1ab2 = json.load(file)

In [171]:
tar = [x['content'] for x in s1ab2]
results = smodel.similarity(tar, list(fdf['sent'].values))

100%|██████████| 2/2 [00:05<00:00,  2.52s/it]


In [172]:
rdf = []
for rind, res in enumerate(results):
    
    sortedIdxs = list(np.argsort(res))
    sortedIdxs.reverse()
    highest = sortedIdxs[0]
    
    row = fdf.iloc[highest]
    
    found = row['found']
    sent = row['sent']
    score = res[highest]
    
    if score < .2:
        continue
    
    o = s1ab2[rind]
    correct = 0
    if o['polarity'] == row['tribe'].lower():
        correct = 1
    tmp = [o['content'], o['polarity'], row['tribe'].lower(), found, sent, score, correct]
    rdf.append(tmp)

pd.set_option('display.max_colwidth', None)
rdf = pd.DataFrame(rdf)
rdf.columns = ['content', 'polarity', 'tribe', 'found', 'sent', 'score', 'correct']
print(rdf['correct'].mean())
rdf

0.6904761904761905


,content,polarity,tribe,found,sent,score,correct
0,"Actor Brad Pitt: 'Trump Is Not My President, We Have No Future With This...'",l,l,fairness,A Republican is acting unfairly,0.314967,1
1,Rice told investigators why she unmasked Trump aides,l,c,loyalty,A Democrat is being disloyal,0.279182,0
2,"MITT ROMNEY: 'TRUMP HAS SOME VERY SERIOUS EMOTIONAL PROBLEMS, HE MUST RESIGN BEFORE WE...'",l,l,care,A Republican is emotionally hurting another person,0.427181,1
3,"OFFICIAL: Rudy Giuliani Officially Resigns, Trump's Legal Team on Its Last Leg",l,l,loyalty,A Republican leaves their group and joins their rival,0.291101,1
4,Obama Announces Bid To Become UN Secretary General - Usa News,l,l,loyalty,A Democrat is being patriotic,0.389628,1
5,Lady Gaga Reveals Plan To Cover Her Face Again 'Until Trump Gets Fired From Office And I Feel Free Once More',l,l,loyalty,A Republican is betraying their group,0.236843,1
6,Starbucks Just Placed Trump's Photo on Floor so Every Customer have to Step On It....,l,l,sanctity,A Democrat protects something sacred,0.216593,1
7,Lindsey Graham Threatens To Convert To Democrat If Republicans Don't Dump Trump - True Trumpers,l,l,loyalty,A Republican is betraying their group,0.526922,1
8,"Obama, Not Donald Trump, May Be Invited to Royal Wedding of Prince Harry and Meghan Markle",l,c,fairness,A Democrat is unfairly getting preferential treatment,0.374645,0
9,"Michelle Obama: 'Florida Shooting Is Clearly Trump's Fault, These Shootings Are Happening Constantly Since He Became Our President.We Must Protect Our Children From This Tyrant'Do You Support Her?'",l,c,fairness,A Democrat is acting unfairly,0.362785,0


### Farch 1

In [170]:
farchNeg = {
    'care': ["A $SUBJECT is physically hurting another person",
             "A $SUBJECT is insulting another person",
             "A $SUBJECT is ridiculing another person",
             "A $SUBJECT is hurting an animal",
             "A $SUBJECT is emotionally hurting another person"],
    'fairness': ["A $SUBJECT is abusing their power",
                 "A $SUBJECT is cheating to win",
                 "A $SUBJECT is unfairly getting preferential treatment",
                 "A $SUBJECT is acting unfairly",
                 "A $SUBJECT is making an unequal judgement",
                 "A $SUBJECT is biased"],
    'loyalty': ["A $SUBJECT is betraying their group",
                "A $SUBJECT is being unpatriotic",
                "A $SUBJECT is being disloyal",
                "A $SUBJECT leaves their group and joins their rival"],
    'authority': ["A $SUBJECT disrespects an authority figure",
                  "A $SUBJECT breaks the rules",
                  "A $SUBJECT insults an authority figure"],
    'sanctity': ["A $SUBJECT ruins something",
                 "A $SUBJECT is lustful",
                 "A $SUBJECT is depraved",
                 "A $SUBJECT is sacrilegious",
                 "A $SUBJECT does something disgusting",
                 # "A $SUBJECT is unholy"
                ],
}

farchPos = {
    'care': ["A $SUBJECT is caring for another person",
             "A $SUBJECT is showing compassion",
             # "A $SUBJECT is insulting a $TARGET"
            ],
    'fairness': ["A $SUBJECT is being fair",
                 "A $SUBJECT is protecting rights"
                ],
    'loyalty': ["A $SUBJECT is being patriotic",
                "A $SUBJECT is being loyal",
                # "A $TARGET is betraying a $TARGET"
               ],
    'authority': ["A $SUBJECT respects an authority figure",
                  "A $SUBJECT obeys the law",
                  # "A $SUBJECT disrespects a $TARGET"
                 ],
    'sanctity': ["A $SUBJECT protects something sacred",
                 "A $SUBJECT keeps something pure",
                 # "A $SUBJECT is holy"
                ],
}

fdf = []
for key in farchNeg:
    for s in farchNeg[key]:
        valence = 'neg'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat')
        }
        fdf.append(r)
    for s in farchPos[key]:
        valence = 'pos'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat').replace("$TARGET", 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican').replace("$TARGET", 'Democrat')
        }
        fdf.append(r)
fdf = pd.DataFrame(fdf)

### Farch 2

In [96]:
farchNeg = {
    'care': ["A $SUBJECT is physically hurting another person",
             "A $SUBJECT is insulting another person",
             "A $SUBJECT is ridiculing another person",
             "A $SUBJECT is hurting an animal",
             "A $SUBJECT is emotionally hurting another person"],
    'fairness': ["A $SUBJECT is abusing their power",
                 "A $SUBJECT is cheating to win",
                 "A $SUBJECT is unfairly getting preferential treatment",
                 "A $SUBJECT is acting unfairly",
                 "A $SUBJECT is making an unequal judgement",
                 "A $SUBJECT is biased"],
    'loyalty': ["A $SUBJECT is betraying their group",
                "A $SUBJECT is being unpatriotic",
                "A $SUBJECT is being disloyal",
                "A $SUBJECT leaves their group and joins their rival"],
    'authority': ["A $SUBJECT disrespects an authority figure",
                  "A $SUBJECT breaks the rules",
                  "A $SUBJECT insults an authority figure",
                  "A $SUBJECT is breaking the law"],
    'sanctity': ["A $SUBJECT ruins something",
                 "A $SUBJECT is lustful",
                 "A $SUBJECT is depraved",
                 "A $SUBJECT is sacrilegious",
                 "A $SUBJECT does something disgusting",
                 "A $SUBJECT is unholy"],
}

farchPos = {
    'care': ["A $SUBJECT is caring for another person",
             "A person is caring for a $SUBJECT"
             "A $SUBJECT is showing compassion",
             "A $SUBJECT is insulting a $TARGET",
             "A person is emotionally hurting a $SUBJECT"],
    'fairness': ["A $SUBJECT is being fair",
                 "A $SUBJECT is protecting rights"],
    'loyalty': ["A $SUBJECT is being patriotic",
                "A $SUBJECT is being loyal",
                "A $TARGET is betraying a $TARGET",
                "A person is betraying a $SUBJECT"],
    'authority': ["A person respects a $SUBJECT authority figure",
                  "A $SUBJECT obeys the law",
                  "A $SUBJECT disrespects a $TARGET authority figure"],
    'sanctity': ["A $SUBJECT protects something sacred",
                 "A $SUBJECT keeps something pure",
                 "A $SUBJECT is holy"],
}

fdf = []
for key in farchNeg:
    for s in farchNeg[key]:
        valence = 'neg'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat')
        }
        fdf.append(r)
    for s in farchPos[key]:
        valence = 'pos'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat').replace("$TARGET", 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican').replace("$TARGET", 'Democrat')
        }
        fdf.append(r)
fdf = pd.DataFrame(fdf)

### Farch 3 MFQ

In [167]:
# farchNeg = {
#     'care': ["someone suffered emotionally ",
#              "someone was cruel",
#             ],
#     'fairness': ["some people were treated differently than others",
#                  "someone acted unfairly",
#                 ],
#     'loyalty': ["someone did something to betray his or her group",
#                 "someone showed a lack of loyalty",
#                ],
#     'authority': ["someone showed a lack of respect for authority",
#                   "an action caused chaos or disorder"
#                  ],
#     'sanctity': ["someone violated standards of purity and decency",
#                  "someone did something disgusting",
#                 ],
# }

# farchPos = {
#     'care': ["someone cared for someone weak or vulnerable",
#             ],
#     'fairness': ["someone was denied his or her rights",
#                 ],
#     'loyalty': ["someone’s action showed love for his or her country",
#                ],
#     'authority': ["someone conformed to the traditions of society",
#                  ],
#     'sanctity': ["someone acted in a way that God would approve of",
#                 ],
# }

farchNeg = {
    'care': ["$SUBJECT suffered emotionally ",
             "$SUBJECT was cruel",
            ],
    'fairness': ["$SUBJECT was treated differently than others",
                 "$SUBJECT acted unfairly",
                ],
    'loyalty': ["$SUBJECT did something to betray his or her group",
                "$SUBJECT showed a lack of loyalty",
               ],
    'authority': ["$SUBJECT showed a lack of respect for authority",
                  "$SUBJECT caused chaos or disorder"
                 ],
    'sanctity': ["$SUBJECT violated standards of purity and decency",
                 "$SUBJECT did something disgusting",
                ],
}

farchPos = {
    'care': ["$SUBJECT cared for someone weak or vulnerable",
            ],
    'fairness': ["$SUBJECT was denied his or her rights",
                ],
    'loyalty': ["$SUBJECT’s action showed love for his or her country",
               ],
    'authority': ["$SUBJECT conformed to the traditions of society",
                 ],
    'sanctity': ["$SUBJECT acted in a way that God would approve of",
                ],
}

fdf = []
for key in farchNeg:
    for s in farchNeg[key]:
        valence = 'neg'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat')
        }
        fdf.append(r)
    for s in farchPos[key]:
        valence = 'pos'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat').replace("$TARGET", 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican').replace("$TARGET", 'Democrat')
        }
        fdf.append(r)
fdf = pd.DataFrame(fdf)

## Moral Foundation Vignettes

In [2]:
with open('mfv_train.json') as file:
    mfvs = json.load(file)

In [52]:
farch = {
    'care': ['A person hurts another person', 'A person harms someone', 'A person is cruel', 'A person is uncaring'],
    'fairness': ['A person is unfair', 'A person is unjust', 'A person treats people unequally', 'A person cheats'],
    'loyalty': ['A person is disloyal', 'A person betrays the group', 'A person is a traitor', 'A person is treasonous'],
    'authority': ['A person causes chaos', 'A person breaks the law', 'A person disrespects authority',
                  'A person violates traditions'],
    'sanctity': ['A person is disgusting', 'A person acts unnaturally', 'A person is depraved', 'A person is impure']
}

farch2 = {
    'care': ["A person is physically hurting another person",
             "A person is insulting another person",
             "A person is ridiculing another person",
             "A person is hurting an animal",
             "A person is emotionally hurting another person"],
    'fairness': ["A person is abusing their power",
                 "A person is cheating to win",
                 "A person is unfairly getting preferential treatment",
                 "A person is acting unfairly",
                 "A person is making an unequal judgement",
                 "A person is acting with bias"],
    'loyalty': ["A person is betraying their group",
                "A person is being unpatriotic",
                "A person is being disloyal",
                "A person leaves their group and joins their rival"],
    'authority': ["A person disrespects an authority figure",
                  "A person breaks the rules",
                  "A person insults an authority figure"],
    'sanctity': ["A person ruins something",
                 "A person is lustful",
                 "A person is depraved",
                 "A group is sacrilegious",
                 "A person does something disgusting",
                 "A person commits a sin"],
}

farch3 = {
    'care': ["A $SUBJECT is physically hurting another person",
             "A $SUBJECT is insulting another person",
             "A $SUBJECT is ridiculing another person",
             "A $SUBJECT is hurting an animal",
             "A $SUBJECT is emotionally hurting another person",
             "A $SUBJECT is caring for another person",
             "A $SUBJECT is showing compassion"],
    'fairness': ["A $SUBJECT is abusing their power",
                 "A $SUBJECT is cheating to win",
                 "A $SUBJECT is unfairly getting preferential treatment",
                 "A $SUBJECT is acting unfairly",
                 "A $SUBJECT is making an unequal judgement",
                 "A $SUBJECT is biased",
                 "A $SUBJECT is being fair",
                 "A $SUBJECT is protecting rights"],
    'loyalty': ["A $SUBJECT is betraying their group",
                "A $SUBJECT is being unpatriotic",
                "A $SUBJECT is being disloyal",
                "A $SUBJECT leaves their group and joins their rival",
                "A $SUBJECT is being patriotic",
                "A $SUBJECT is being loyal"],
    'authority': ["A $SUBJECT disrespects an authority figure",
                  "A $SUBJECT breaks the rules",
                  "A $SUBJECT insults an authority figure",
                  "A $SUBJECT respects an authority figure",
                  "A $SUBJECT obeys the law"],
    'sanctity': ["A $SUBJECT ruins something",
                 "A $SUBJECT is lustful",
                 "A $SUBJECT is depraved",
                 "A $SUBJECT is sacrilegious",
                 "A $SUBJECT does something disgusting",
                 "A $SUBJECT commits a sin",
                 "A $SUBJECT protects something sacred",
                 "A $SUBJECT keeps something pure",
                 "A $SUBJECT is holy"],
}

farchParty = {}
for key in farch3:
    farchParty[key] = []
    for s in farch3[key]:
        farchParty[key].append(s.replace('$SUBJECT', 'person'))
        farchParty[key].append(s.replace('$SUBJECT', 'Democrat'))
        farchParty[key].append(s.replace('$SUBJECT', 'Republican'))

farch = mfc.mfd2
farch = farchParty

archs = []
archsf = []
for key in farch:
    archs = archs+farch[key]
    for el in farch[key]:
        archsf.append(key)

def bestMatch(s):
    bestScore = 0
    bestFound = ""
    bestArch = ""
    for key in farch:
        sims = smodel.similarity([s], farch[key])
        for sind, sim in enumerate(sims[0]):
            if sim > bestScore:
                bestScore = sim
                bestFound = key
                bestArch = farch[key][sind]
    return bestFound, bestScore, bestArch

nonLiberty = list(filter(lambda x: x['Foundation'] != 'Liberty', mfvs))

tar = [x['text'] for x in nonLiberty]
results = smodel.similarity(tar, archs)

100%|██████████| 2/2 [00:02<00:00,  1.17s/it]


In [49]:
corrects = 0
total = 0
fcorrects = {}
for key in farch:
    fcorrects[key] = [0, 0]
for rind, res in enumerate(results):
    
    sortedIdxs = np.argsort(res)
    
    # max_ind = list(res).index(max(res))
    # found = archsf[max_ind]
    found = archsf[sortedIdxs[-1]]
    
    vin = nonLiberty[rind]
    
    # if res[sortedIdxs[-1] < .2:
    
    fcorrects[found.lower()][1] += 1
    total += 1
    if found.lower() == vin['Foundation'].lower():
        corrects += 1
        fcorrects[found.lower()][0] += 1
    else:
        print(f"{vin['text']} ({vin['Foundation']}) : {archs[sortedIdxs[-1]]} ({found} = {res[sortedIdxs[-1]]})")
#     for si in sortedIdxs[-3:-1]:
#         print(f"{archs[si]} ({archsf[si]} = {res[si]})")
#     print()
        
print(corrects/len(nonLiberty))

for key in fcorrects:
    print(f"{key}: {(fcorrects[key][0]/fcorrects[key][1])}")

You see a mayor saying that the neighboring town is a much better town. (Loyalty) : A Democrat respects an authority figure (authority = 0.37769564986228943)
You see a Hollywood star agreeing with a foreign dictator's denunciation of the US. (Loyalty) : A Democrat is ridiculing another person (care = 0.42366746068000793)
You see a boy skipping to the front of the line because his friend is an employee. (Fairness) : A person leaves their group and joins their rival (loyalty = 0.3494332730770111)
You see a teaching assistant talking back to the teacher in front of the classrom. (Authority) : A person is insulting another person (care = 0.38782772421836853)
You see a story about a remote tribe eating the flesh of their deceased members. (Sanctity) : A person is betraying their group (loyalty = 0.27945029735565186)
You see the class president saying on TV that her rival college is a better school. (Loyalty) : A Democrat is biased (fairness = 0.3381158411502838)
You see a boy turning up the

## Persuasion Invasion Scenarios

In [70]:
with open('PI_scenarioData.json') as file:
    pis = json.load(file)
    
trimmed = []
tmp = []
for x in pis:
    if x['text'] not in tmp:
        tmp.append(x['text'])
        trimmed.append(x)
print(len(pis))
pis = trimmed
print(len(pis))

81
69


In [71]:
farch = {
    'care': ['A person hurts another person', 'A person harms someone', 'A person is cruel', 'A person is uncaring'],
    'fairness': ['A person is unfair', 'A person is unjust', 'A person treats people unequally', 'A person cheats'],
    'loyalty': ['A person is disloyal', 'A person betrays the group', 'A person is a traitor', 'A person is treasonous'],
    'authority': ['A person causes chaos', 'A person breaks the law', 'A person disrespects authority',
                  'A person violates traditions'],
    'sanctity': ['A person is disgusting', 'A person acts unnaturally', 'A person is depraved', 'A person is impure']
}

archs = []
archsf = []
for key in farch:
    archs = archs+farch[key]
    for el in farch[key]:
        archsf.append(key)

def bestMatch(s):
    bestScore = 0
    bestFound = ""
    bestArch = ""
    for key in farch:
        sims = smodel.similarity([s], farch[key])
        for sind, sim in enumerate(sims[0]):
            if sim > bestScore:
                bestScore = sim
                bestFound = key
                bestArch = farch[key][sind]
    return bestFound, bestScore, bestArch

tar = [x['text'] for x in pis]
results = smodel.similarity(tar, archs)

100%|██████████| 1/1 [00:00<00:00,  2.49it/s]


In [73]:
corrects = 0
for rind, res in enumerate(results):
    max_ind = list(res).index(max(res))
    found = archsf[max_ind]
    vin = pis[rind]
    if found.lower() == vin['Foundation'].lower():
        corrects += 1
        
    # else:
        print(f"{vin['text']} ({vin['Foundation']}) : {archs[max_ind]} ({found})")
print(corrects/len(pis))

It's not fair that we have to pay for the uniforms. (fairness) : A person is unfair (fairness)
I heard that kids who wear uniforms are more likely to break the rules. (authority) : A person breaks the law (authority)
Having uniforms puts everyone on the same playing field, which is more fair. (fairness) : A person treats people unequally (fairness)
Under the current laws, people should already not be using fireworks in any public place. (authority) : A person violates traditions (authority)
We've been setting off fireworks in campgrounds for decades. (authority) : A person causes chaos (authority)
Paid tutoring gives an unfair advantage to rich students. (fairness) : A person is unfair (fairness)
The school rules state that each individual tutoring session has to be approved by the principal. Seems like a lot of work. (authority) : A person violates traditions (authority)
Every child should have the same access to high quality parks. (fairness) : A person treats people unequally (fairn

## Politifact Fake News Dataset

In [175]:
farchNeg = {
    'care': ["A $SUBJECT is physically hurting another person",
             "A $SUBJECT is insulting another person",
             "A $SUBJECT is ridiculing another person",
             "A $SUBJECT is hurting an animal",
             "A $SUBJECT is emotionally hurting another person"],
    'fairness': ["A $SUBJECT is abusing their power",
                 "A $SUBJECT is cheating to win",
                 "A $SUBJECT is unfairly getting preferential treatment",
                 "A $SUBJECT is acting unfairly",
                 "A $SUBJECT is making an unequal judgement",
                 "A $SUBJECT is biased"],
    'loyalty': ["A $SUBJECT is betraying their group",
                "A $SUBJECT is being unpatriotic",
                "A $SUBJECT is being disloyal",
                "A $SUBJECT leaves their group and joins their rival"],
    'authority': ["A $SUBJECT disrespects an authority figure",
                  "A $SUBJECT breaks the rules",
                  "A $SUBJECT insults an authority figure"],
    'sanctity': ["A $SUBJECT ruins something",
                 "A $SUBJECT is lustful",
                 "A $SUBJECT is depraved",
                 "A $SUBJECT is sacrilegious",
                 "A $SUBJECT does something disgusting",
                 # "A $SUBJECT is unholy"
                ],
}

farchPos = {
    'care': ["A $SUBJECT is caring for another person",
             "A $SUBJECT is showing compassion",
             # "A $SUBJECT is insulting a $TARGET"
            ],
    'fairness': ["A $SUBJECT is being fair",
                 "A $SUBJECT is protecting rights"
                ],
    'loyalty': ["A $SUBJECT is being patriotic",
                "A $SUBJECT is being loyal",
                # "A $TARGET is betraying a $TARGET"
               ],
    'authority': ["A $SUBJECT respects an authority figure",
                  "A $SUBJECT obeys the law",
                  # "A $SUBJECT disrespects a $TARGET"
                 ],
    'sanctity': ["A $SUBJECT protects something sacred",
                 "A $SUBJECT keeps something pure",
                 # "A $SUBJECT is holy"
                ],
}

fdf = []
for key in farchNeg:
    for s in farchNeg[key]:
        valence = 'neg'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat')
        }
        fdf.append(r)
    for s in farchPos[key]:
        valence = 'pos'
        r = {
            'valence':valence,
            'tribe':'L',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat').replace("$TARGET", 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican').replace("$TARGET", 'Democrat')
        }
        fdf.append(r)
fdf = pd.DataFrame(fdf)

In [176]:
df = pd.read_csv('S1.AlignmentBelievability/S1.ab.1/politifact_fake_substinative.csv')

In [177]:
df.head()

,id,type,news_url,title
0,politifact11773,fake,http://www.occupydemocrats.com/virginia-republican-wants-schools-to-check-childrens-genitals-before-using-bathroom/,Virginia Republican Wants Schools To Check Children's Genitals Before Using Bathroom
1,politifact13468,fake,http://www.bbc.com/news/election-us-2016-37987306?ocid=socialflow_twitter,Denzel Washington hit by Facebook fake news story on Trump
2,politifact13475,fake,genius.com/discussions/219349-Donald-trump-protester-speaks-out-i-was-paid-s3500-to-protest-trumps-rally,"Donald Trump Protester Speaks Out: ‚I Was Paid $3,500 To Protest Trump‚'s Rally‚"
3,politifact13501,fake,http://therightists.com/lady-gaga-reveals-plan-to-cover-her-face-again-until-trump-gets-fired-from-office-and-i-feel-free-once-more/,Lady Gaga Reveals Plan To Cover Her Face Again ‚Until Trump Gets Fired From Office And I Feel Free Once More‚
4,politifact13541,fake,http://freedomdaily.com/obama-led-u-n-has-just/,BREAKING ObamaLed UN Has Just Made It Official US To Immediately Pay Blacks ‚Reparations‚'


In [179]:
tar = list(df['title'].values)
results = smodel.similarity(tar, list(fdf['sent'].values))

100%|██████████| 2/2 [00:05<00:00,  2.69s/it]


In [187]:
rdf = []
for rind, res in enumerate(results):
    
    sortedIdxs = list(np.argsort(res))
    sortedIdxs.reverse()
    highest = sortedIdxs[0]
    
    row = fdf.iloc[highest]
    second = fdf.iloc[sortedIdxs[1]]
    
    found = row['found']
    sent = row['sent']
    score = res[highest]
    
    if score < .2:
        continue
    
    o = df.iloc[rind]
    tmp = [o['title'], row['tribe'].lower(), found, sent, score, second['found'], second['sent'], res[sortedIdxs[1]] ]
    rdf.append(tmp)

pd.set_option('display.max_colwidth', None)
rdf = pd.DataFrame(rdf)
rdf.columns = ['content', 'tribe', 'found', 'sent', 'score', 'found2', 'sent2', 'score2']
rdf.sort_values(by="score", ascending=False).head(20)

,content,tribe,found,sent,score,found2,sent2,score2
60,Lindsey Graham Threatens To Convert To Democrat If Republicans Don‚'t Dump Trump ‚ True Trumpers,l,loyalty,A Republican is betraying their group,0.526626,loyalty,A Democrat is betraying their group,0.511109
296,Senate Democrats Demand Supreme Court Nominee Not Be Unduly Influenced By U.S. Constitution,l,fairness,A Democrat is protecting rights,0.521261,authority,A Democrat obeys the law,0.503138
12,ELECTORAL COLLEGE ELECTOR COMMITS SUICIDE TO AVOID VOTING FOR DONALD TRUMP,l,loyalty,A Republican is betraying their group,0.519449,fairness,A Republican is cheating to win,0.462041
302,British Politician Beaten by Protestors for 'Welcoming Trump to the UK',l,authority,A Republican insults an authority figure,0.508381,authority,A Democrat insults an authority figure,0.507554
61,BREAKING: Bill O‚'Reilly In Critical Condition After Being Attacked By ‚Tolerant‚' Liberals,l,care,A Republican is emotionally hurting another person,0.505121,care,A Republican is physically hurting another person,0.472369
150,CALIFORNIA DEMOCRAT MAXINE WATERS CHARGED ON 3 COUNTS. DO YOU SUPPORT THIS?,l,fairness,A Democrat is protecting rights,0.484716,fairness,A Democrat is making an unequal judgement,0.448162
105,"Barack Obama Tweets SICK Attack On John McCain, Says He Should Have Died",c,care,A Democrat is insulting another person,0.477451,care,A Democrat is emotionally hurting another person,0.468586
8,BREAKING: PUTIN INTERFERENCE COULD GIVE COURTS LEGAL RIGHT TO MAKE HILLARY PRESIDENT,c,authority,A Democrat breaks the rules,0.474253,fairness,A Democrat is abusing their power,0.409178
91,U.N. Official 'Accidentally' Crushes Own Throat Right Before Testifying Against Hillary Clinton,c,loyalty,A Democrat is being disloyal,0.464941,authority,A Democrat insults an authority figure,0.449186
132,"The pro-Russia, pro-weed, pro-Assange GOP congressman who will be tough to beat",l,fairness,A Republican is biased,0.463233,fairness,A Democrat is biased,0.453094


## Fake Substinative Trimmed

In [229]:
farchNeg = {
    'care': ["A $SUBJECT is physically hurting another person",
             "A $SUBJECT is insulting another person",
             "A $SUBJECT is ridiculing another person",
             "A $SUBJECT is hurting an animal",
             "A $SUBJECT is emotionally hurting another person"],
    'fairness': ["A $SUBJECT is abusing their power",
                 "A $SUBJECT is cheating to win",
                 "A $SUBJECT is unfairly getting preferential treatment",
                 "A $SUBJECT is being unfair",
                 "A $SUBJECT is making an unequal judgement"],
    'loyalty': ["A $SUBJECT is betraying their group",
                "A $SUBJECT is being unpatriotic",
                "A $SUBJECT is being disloyal",
                "A $SUBJECT leaves their group and joins their rival"],
    'authority': ["A $SUBJECT disrespects an authority figure",
                  "A $SUBJECT breaks the rules",
                  "A $SUBJECT insults an authority figure",
                  "A $SUBJECT breaks the law"],
    'sanctity': ["A $SUBJECT ruins something",
                 "A $SUBJECT is lustful",
                 "A $SUBJECT is depraved",
                 "A $SUBJECT is sacrilegious",
                 "A $SUBJECT does something disgusting",
                 # "A $SUBJECT is unholy"
                ]
}

farchPos = {
    'care': ["A $SUBJECT is caring for another person",
             "A $SUBJECT is showing compassion",
             # "A $SUBJECT is insulting a $TARGET"
            ],
    'fairness': ["A $SUBJECT is being fair",
                 "A $SUBJECT is protecting rights"
                ],
    'loyalty': ["A $SUBJECT is being patriotic",
                "A $SUBJECT is being loyal",
                # "A $TARGET is betraying a $TARGET"
               ],
    'authority': ["A $SUBJECT respects an authority figure",
                  "A $SUBJECT obeys the law",
                  "A $TARGET is disrespected",
                  "A $SUBJECT is respected"
                 ],
    'sanctity': ["A $SUBJECT protects something sacred",
                 "A $SUBJECT keeps something pure",
                 # "A $SUBJECT is holy"
                ]
}

fdf = []
for key in farchNeg:
    for s in farchNeg[key]:
        valence = 'neg'
        r = {
            'valence':valence,
            'tribe':'L',
            'subject': 'Republicans',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'subject': 'Democrats',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat')
        }
        fdf.append(r)
    for s in farchPos[key]:
        valence = 'pos'
        r = {
            'valence':valence,
            'tribe':'L',
            'subject': 'Democrats',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Democrat').replace("$TARGET", 'Republican')
        }
        fdf.append(r)
        r = {
            'valence':valence,
            'tribe':'C',
            'subject': 'Republicans',
            'found': key,
            'sent': s.replace('$SUBJECT', 'Republican').replace("$TARGET", 'Democrat')
        }
        fdf.append(r)
fdf = pd.DataFrame(fdf)

In [272]:
def preprocess(dat):
    republicans = ['Donald Trump', "TRUMP", 'Trump', 'Mitch McConnell', 'Ted Cruz', 'Romney', 'Jeb Bush']
    democrats = ['Barack Obama', 'Obama', "BIDEN", "Biden", 'Bernie Sanders', 'Nancy Pelosi', 'Harry Reid', 'Hillary Clinton', 'Clinton']
    for i in dat:
        i['p_title'] = i['title']
        for r in republicans:
            i['p_title'] = i['p_title'].replace(r, 'Republican')
        for d in democrats:
            i['p_title'] = i['p_title'].replace(d, 'Democrat')
    return dat
        

In [273]:
items = [{"newID":"politifact10332_real","title":"Obama: Gitmo inspires jihadists"},{"newID":"politifact10731_real","title":"Call 'Islamic terrorism' what it is: a threat to West, Jeb Bush says"},{"newID":"politifact11770_real","title":"Bernie Sanders Supports Gun Safety Bill"},{"newID":"politifact1185_real","title":"Obama Promises Abortion in Public Plan"},{"newID":"politifact11989_real","title":"Memory Lapse? Trump Seeks Distance From 'Advisor' With Past Ties to Mafia"},{"newID":"politifact12627_real","title":"Trump's Tough Talk on Hedge-Fund Taxes Doesn't Match His Plan"},{"newID":"politifact13229_real","title":"Donald Trump featured in new jihadist recruitment video"},{"newID":"politifact13443_real","title":"Families Facing Tax Increases Under Trump's Tax Plan"},{"newID":"politifact14064_real","title":"Donald Trump exaggerates when he says China has 'total control' over North Korea"},{"newID":"politifact1467_real","title":"Sen. Gregg: President Obama's Budget Proposal Is a 'Disaster'"},{"newID":"politifact1519_real","title":"Harry Reid Says GOP Should \"Stop Crying\" About Reconciliation"},{"newID":"politifact6730_real","title":"SECRET VIDEO: Romney Tells Millionaire Donors What He REALLY Thinks of Obama Voters"},{"newID":"politifact8778_real","title":"Barack Obama says U.S. oil production last year was highest since 2003"},{"newID":"politifact10332_fabReal","title":"Trump: Gitmo inspires jihadists"},{"newID":"politifact10731_fabReal","title":"Call 'Islamic terrorism' what it is: a threat to West, Hillary Clinton says"},{"newID":"politifact11770_fabReal","title":"Ted Cruz Supports Gun Safety Bill"},{"newID":"politifact1185_fabReal","title":"Trump Promises Abortion in Public Plan"},{"newID":"politifact11989_fabReal","title":"Memory Lapse? Obama Seeks Distance From 'Advisor' With Past Ties to Mafia"},{"newID":"politifact12627_fabReal","title":"Obama's Tough Talk on Hedge-Fund Taxes Doesn't Match His Plan"},{"newID":"politifact13229_fabReal","title":"Barack Obama featured in new jihadist recruitment video"},{"newID":"politifact13443_fabReal","title":"Families Facing Tax Increases Under Obama's Tax Plan"},{"newID":"politifact14064_fabReal","title":"Barack Obama exaggerates when he says China has 'total control' over North Korea"},{"newID":"politifact1467_fabReal","title":"Sen. Gregg: President Trump's Budget Proposal Is a 'Disaster'"},{"newID":"politifact1519_fabReal","title":"Mitch McConnell Says Dems Should \"Stop Crying\" About Reconciliation"},{"newID":"politifact6730_fabReal","title":"SECRET VIDEO: Obama Tells Millionaire Donors What He REALLY Thinks of Romney Voters"},{"newID":"politifact8778_fabReal","title":"Donald Trump says U.S. oil production last year was highest since 2003"},{"newID":"politifact13929_fake","title":"BREAKING: Nancy Pelosi Was Just Taken From Her Office In Handcuffs"},{"newID":"politifact13987_fake","title":"Obama's Tax- Skipping ' Audit Shows Millions In Offshore Accounts"},{"newID":"politifact14021_fake","title":"Trump on Revamping the Military: We're Bringing Back the Draft"},{"newID":"politifact14126_fake","title":"Breaking News: FBI Uncovers Evidence That 62 Million Trump Voters are All Russian Agents"},{"newID":"politifact14293_fake","title":"Spicer: 'Trump Has The Legal Right To Cancel SCOTUS Because They Serve At His Pleasure'"},{"newID":"politifact14406_fake","title":"Trump Orders 13 Obama Deep State Operatives Arrested For Treason"},{"newID":"politifact14500_fake","title":"Clinton Foundations Sends Water To Houston For $7 A Bottle"},{"newID":"politifact14810_fake","title":"BREAKING: Federal Judge Grants Permission To Subpoena Trump"},{"newID":"politifact15156_fake","title":"Court Orders Obama To Pay $400 Million In Restitution"},{"newID":"politifact15224_fake","title":"Obama Secretly Flees US ' Leaves Stunning Evidence Behind"},{"newID":"politifact15287_fake","title":"BREAKING: British Intelligence Seizes Clinton Foundation Warehouse, $400 Million In Cash"},{"newID":"politifact15309_fake","title":"BREAKING: Nancy Pelosi To Resign After New Revelations"},{"newID":"politifact15501_fake","title":"TRUMP APPROVAL RATING Better than Obama and Reagan at Same Point in their Presidencies"},{"newID":"politifact13929_fabFake","title":"BREAKING: Mitch McConnell Was Just Taken From His Office In Handcuffs"},{"newID":"politifact13987_fabFake","title":"Trump's Tax- Skipping ' Audit Shows Millions In Offshore Accounts"},{"newID":"politifact14021_fabFake","title":"Obama on Revamping the Military: We're Bringing Back the Draft"},{"newID":"politifact14126_fabFake","title":"Breaking News: FBI Uncovers Evidence That 62 Million Clinton Voters are All Russian Agents"},{"newID":"politifact14293_fabFake","title":"Gibbs: 'Obama Has The Legal Right To Cancel SCOTUS Because They Serve At His Pleasure'"},{"newID":"politifact14406_fabFake","title":"Biden Orders 13 Trump Deep State Operatives Arrested For Treason"},{"newID":"politifact14500_fabFake","title":"Trump Foundations Sends Water To Houston For $7 A Bottle"},{"newID":"politifact14810_fabFake","title":"BREAKING: Federal Judge Grants Permission To Subpoena Obama"},{"newID":"politifact15156_fabFake","title":"Court Orders Trump To Pay $400 Million In Restitution"},{"newID":"politifact15224_fabFake","title":"Trump Secretly Flees US ' Leaves Stunning Evidence Behind"},{"newID":"politifact15287_fabFake","title":"BREAKING: British Intelligence Seizes Trump Foundation Warehouse, $400 Million In Cash"},{"newID":"politifact15309_fabFake","title":"BREAKING: Mitch McConnell To Resign After New Revelations"},{"newID":"politifact15501_fabFake","title":"BIDEN APPROVAL RATING Better than Trump and Reagan at Same Point in their Presidencies"}]
items = preprocess(items)
results = smodel.similarity([x['p_title'] for x in items], list(fdf['sent'].values))

100%|██████████| 2/2 [00:05<00:00,  2.58s/it]


In [274]:
fd = items.copy()
for rind, res in enumerate(results):
    
    sortedIdxs = list(np.argsort(res))
    sortedIdxs.reverse()
    highest = sortedIdxs[0]
    
    row = fdf.iloc[highest]
    
    found = row['found']
    sent = row['sent']
    score = res[highest]
    
    fd[rind]['simcse_foundation'] = found
    fd[rind]['simcse_sentence'] = sent
    fd[rind]['simcse_score'] = score
    fd[rind]['simcse_subject'] = row['subject']
    fd[rind]['simcse_tribe'] = row['tribe'].lower()
    fd[rind]['simcse_valence'] = row['valence']

In [278]:
pd.DataFrame(fd).sort_values('simcse_score', ascending=False).to_json('ab3_stimuli.json', orient='records')

In [275]:
pd.DataFrame(fd).sort_values('simcse_score', ascending=False)

,newID,title,p_title,simcse_foundation,simcse_sentence,simcse_score,simcse_subject,simcse_tribe,simcse_valence
33,politifact14810_fake,BREAKING: Federal Judge Grants Permission To Subpoena Trump,BREAKING: Federal Judge Grants Permission To Subpoena Republican,authority,A Republican breaks the law,0.626743,Republicans,l,neg
46,politifact14810_fabFake,BREAKING: Federal Judge Grants Permission To Subpoena Obama,BREAKING: Federal Judge Grants Permission To Subpoena Democrat,authority,A Democrat breaks the law,0.621934,Democrats,c,neg
15,politifact11770_fabReal,Ted Cruz Supports Gun Safety Bill,Republican Supports Gun Safety Bill,fairness,A Republican is protecting rights,0.614500,Republicans,c,pos
50,politifact15309_fabFake,BREAKING: Mitch McConnell To Resign After New Revelations,BREAKING: Republican To Resign After New Revelations,loyalty,A Republican is betraying their group,0.609312,Republicans,l,neg
37,politifact15309_fake,BREAKING: Nancy Pelosi To Resign After New Revelations,BREAKING: Democrat To Resign After New Revelations,loyalty,A Democrat is betraying their group,0.588800,Democrats,c,neg
11,politifact6730_real,SECRET VIDEO: Romney Tells Millionaire Donors What He REALLY Thinks of Obama Voters,SECRET VIDEO: Republican Tells Millionaire Donors What He REALLY Thinks of Democrat Voters,fairness,A Republican is cheating to win,0.585451,Republicans,l,neg
39,politifact13929_fabFake,BREAKING: Mitch McConnell Was Just Taken From His Office In Handcuffs,BREAKING: Republican Was Just Taken From His Office In Handcuffs,authority,A Republican breaks the law,0.584489,Republicans,l,neg
4,politifact11989_real,Memory Lapse? Trump Seeks Distance From 'Advisor' With Past Ties to Mafia,Memory Lapse? Republican Seeks Distance From 'Advisor' With Past Ties to Mafia,loyalty,A Republican is being disloyal,0.575153,Republicans,l,neg
30,politifact14293_fake,Spicer: 'Trump Has The Legal Right To Cancel SCOTUS Because They Serve At His Pleasure',Spicer: 'Republican Has The Legal Right To Cancel SCOTUS Because They Serve At His Pleasure',fairness,A Republican is unfairly getting preferential treatment,0.573773,Republicans,l,neg
38,politifact15501_fake,TRUMP APPROVAL RATING Better than Obama and Reagan at Same Point in their Presidencies,Republican APPROVAL RATING Better than Democrat and Reagan at Same Point in their Presidencies,authority,A Republican is respected,0.567992,Republicans,c,pos
